# Dataset Preparation

In [1]:
import numpy as np
import pandas as pd

# 1. Import Dataset

In [12]:
master_table = pd.read_csv('data/master_table_991patients_09_27_21.csv') # patient information/outcomes
med_data = pd.read_csv('data/med_admin.csv') # medication record
df_med_list = pd.read_csv("data/Emory Meds 10.15.21.csv") # metadata on medications
df_med_list.loc[:, "ROUTE"] = df_med_list["ROUTE"].str.replace('\W', '')
df_med_list.loc[:, "ROUTE"] = df_med_list["ROUTE"].str.upper()
med_dict = dict(zip(df_med_list.DESCRIPTION.str.upper().tolist(),df_med_list.FIRST_NAME))
route_dict = dict(zip(df_med_list.DESCRIPTION.str.upper().tolist(), df_med_list.ROUTE))

/var/folders/54/1zrdd3f502z54thfc2d33g580000gn/T/ipykernel_73985/2727333952.py:2: DtypeWarning: Columns (6,18) have mixed types. Specify dtype option on import or set low_memory=False.
  med_data = pd.read_csv('data/med_admin.csv')


In [13]:
# columns in the medication dataset
med_data.columns

Index(['PATIENT_DEID', 'ENCOUNTER_DEID', 'ORDERING_DATE_DEID', 'MEDICATION_ID',
       'DESCRIPTION', 'QUANTITY', 'REFILLS', 'START_DATE_DEID',
       'END_DATE_DEID', 'NAME', 'THERA_CLASS_NAME', 'PHARM_CLASS_NAME',
       'PHARM_SUBCLASS_NAME', 'GENERIC_NAME', 'STRENGTH', 'FORM', 'ROUTE',
       'FREQ_NAME', 'ORDER_DOSE', 'ORDER_DOSE_UNIT_NAME', 'MAR_LINE',
       'MAR_TAKEN_TIME_DEID', 'MAR_ACTION_NAME', 'MAR_ROUTE_NAME'],
      dtype='object')

In [14]:
print("number of unique patients: {}".format(med_data.PATIENT_DEID.nunique()))
print("number of medication records: {}".format(len(med_data)))

number of unique patients: 1000
number of medication records: 630364


# 2. Preprocessing

## 2.1 Medication data

In [15]:
# Removing rows where the "DESCRIPTION" column is NaN (missing)
med_data = med_data.dropna(subset=["DESCRIPTION"])

# Converting the strings in the "DESCRIPTION" column to uppercase
med_data["DESCRIPTION"] = med_data["DESCRIPTION"].str.upper()

# Mapping the "DESCRIPTION" to "FIRST_NAME" using a dictionary (med_dict)
med_data["FIRST_NAME"] = med_data["DESCRIPTION"].map(med_dict)

# Mapping the "DESCRIPTION" to "ROUTE" using a dictionary (route_dict)
med_data["ROUTE"] = med_data["DESCRIPTION"].map(route_dict)

# Removing rows where the "FIRST_NAME" column is NaN (missing)
med_data = med_data.dropna(subset=["FIRST_NAME"])

# List of medication actions that are considered as 'given'
med_given = ["Given", "New Bag", "Restarted", "Rate Change"]

# Filtering the data to include only the rows where "MAR_ACTION_NAME" 
# is in the 'med_given' list, and selecting specific columns to keep
med_data = med_data[med_data["MAR_ACTION_NAME"].isin(med_given)][
    ["PATIENT_DEID", "MAR_TAKEN_TIME_DEID", "DESCRIPTION", "MEDICATION_ID", "FIRST_NAME", "ROUTE"]
]


In [16]:
# Removing non-medication/free-notes that are irrelavent

meds_to_remove = ["ABCIXIMAB INFUSION", 
    "ZZ IMS TEMPLATE",
    "BLOOD SUGAR DIAGNOSTIC STRIPS",
    "INFUSION BAG BUILDER (ALL ROUTES AVAILABLE)",
    "PHARMACY COMMUNICATION PHARMACY DOSING ANTIBIOTIC",
    "NON FORMULARY",
    "PHARMACY COMMUNICATION - LIQUID/SOLID MEDICATIONS (REX)",
    "CHEMO CLARIFICATION ORDER",
    "NON FORMULARY",
    "ADULT 3-IN-1 TPN UNC TEMPLATE",
    "PET RB-82",
    "OR SMALL VOLUME BUILDER",
    "OR IRRIGATION BUILDER",
    "OKAY TO SEND MEDICATION/CHEMOTHERAPY TO UNIT",
    "OR SMALL VOLUME BUILDER (UNC)",
    "IP OKAY TO TREAT",
    "BLOOD-GLUCOSE METER KIT",
    "BLOOD SUGAR DIAGNOSTIC STRIPS",
    "LANCETS",
    "MINERAL OIL",
    "BLOOD-GLUCOSE METER",
    "WATER FOR IRRIGATION, STERILE SOLUTION",
    "WHITE PETROLATUM",
    "VACCINE",
    'TPN', 'PET RB-82', 'OR SMALL VOLUME BUILDER',
       'OR IRRIGATION BUILDER', 'OR SMALL VOLUME BUILDER (UNC)', "ADULT CUSTOM IV INFUSION BUILDER (HELP)"]

med_data = med_data[~med_data['FIRST_NAME'].isin(meds_to_remove)]



## 2.2 Master table

In [18]:
# Selecting specific columns from the master_table to create a new dataframe, selected_feature
selected_feature = master_table[
    ["PATIENT_DEID", "ICU_stay_N", "age_at_time", "race_comb", "score_24h", "Death_at_discharge",
     "Delirium_ICU", "AKI", "total_length_days", "Mechanical_ventilation", "dur_MV_days",
     "admission_dx", "first_MV", "firstMV_fac", "vasopressor_ICU_day"]
].copy()

# DURATION FILLING
# Replacing NaN values in 'dur_MV_days' and 'vasopressor_ICU_day' columns with 0
selected_feature.loc[:, "dur_MV_days"] = selected_feature["dur_MV_days"].fillna(0)
selected_feature.loc[:, "vasopressor_ICU_day"] = selected_feature["vasopressor_ICU_day"].fillna(0)

# Replacing NaN values in 'total_length_days' and 'first_MV' columns with 0, using dot notation for accessing columns
selected_feature.total_length_days = selected_feature.total_length_days.fillna(0)
selected_feature.first_MV = selected_feature.first_MV.fillna(0)

# Replacing any remaining NaN values in the dataframe with "Unknown"
selected_feature = selected_feature.fillna("Unknown")


In [20]:
"""
# ALL BINARIZE
selected_feature = selected_feature.fillna(0)
selected_features = selected_feature.replace("No", 0)
selected_features = selected_feature.replace("Yes", 1)
selected_features = selected_feature.replace("Negative", 0)
selected_features = selected_feature.replace("Positive", 1)
selected_features.firstMV_fac = selected_features.firstMV_fac.replace("none", -1)
selected_features.firstMV_fac = selected_features.firstMV_fac.replace("before 24h", 0)
selected_features.firstMV_fac = selected_features.firstMV_fac.replace("after 24h", 1)

selected_feature.loc[selected_feature["Death_at_discharge"] == "dead_at_discharge", "Death_at_discharge"] = 1
selected_feature.loc[selected_feature["Death_at_discharge"] == "alive", "Death_at_discharge"] = 0
"""

'\n# ALL BINARIZE\nselected_feature = selected_feature.fillna(0)\nselected_features = selected_feature.replace("No", 0)\nselected_features = selected_feature.replace("Yes", 1)\nselected_features = selected_feature.replace("Negative", 0)\nselected_features = selected_feature.replace("Positive", 1)\nselected_features.firstMV_fac = selected_features.firstMV_fac.replace("none", -1)\nselected_features.firstMV_fac = selected_features.firstMV_fac.replace("before 24h", 0)\nselected_features.firstMV_fac = selected_features.firstMV_fac.replace("after 24h", 1)\n\nselected_feature.loc[selected_feature["Death_at_discharge"] == "dead_at_discharge", "Death_at_discharge"] = 1\nselected_feature.loc[selected_feature["Death_at_discharge"] == "alive", "Death_at_discharge"] = 0\n'

# 3. Create dataset

## 3.1 Dataset with route info

In [23]:
# Copying specific columns from med_data to create a new dataframe, temp
temp = med_data[["PATIENT_DEID", "FIRST_NAME", "ROUTE"]].copy()

# Creating a new column 'FIRST_NAME_ROUTE' by concatenating 'FIRST_NAME' and 'ROUTE' columns with a space in between
temp.loc[:, "FIRST_NAME_ROUTE"] = temp.FIRST_NAME + " " + temp.ROUTE

# Dropping the now redundant 'FIRST_NAME' and 'ROUTE' columns
temp = temp.drop(["FIRST_NAME", "ROUTE"], axis=1)

# Removing duplicate rows from temp
temp = temp.drop_duplicates()

In [25]:
# Grouping the data by 'PATIENT_DEID', and aggregating the unique medications ('FIRST_NAME_ROUTE') into lists
meds_per_pt = temp.groupby("PATIENT_DEID")["FIRST_NAME_ROUTE"].apply(list).reset_index(name="GIVEN_MEDS")

In [27]:
# Importing TransactionEncoder from mlxtend.preprocessing
from mlxtend.preprocessing import TransactionEncoder

# Converting 'GIVEN_MEDS' column values into a list of lists
aki_meds = meds_per_pt.GIVEN_MEDS.tolist()

# Initializing TransactionEncoder and transforming the data
te = TransactionEncoder()
te_ary = te.fit(aki_meds).transform(aki_meds)

# Creating a new DataFrame from the transformed array
df = pd.DataFrame(te_ary, columns=te.columns_)

In [28]:
# Adding 'PATIENT_DEID' and 'GIVEN_MEDS' columns to the new DataFrame
df["PATIENT_DEID"] = meds_per_pt["PATIENT_DEID"]
df["GIVEN"] = meds_per_pt["GIVEN_MEDS"]

# Merging the new DataFrame with the selected_feature DataFrame based on 'PATIENT_DEID'
vec_med_features = pd.merge(df, selected_feature, on="PATIENT_DEID")

In [31]:
vec_med_features.to_csv("vec_med_features_1029.csv", index = False)

## 3.2 Dataset *without* route info

In [33]:
temp = med_data[["PATIENT_DEID", "FIRST_NAME"]].copy()

# Removing duplicate rows from temp
temp = temp.drop_duplicates()

# Grouping the data by 'PATIENT_DEID' and aggregating the unique medications ('FIRST_NAME') into lists
meds_per_pt = temp.groupby("PATIENT_DEID")["FIRST_NAME"].apply(list).reset_index(name="GIVEN_MEDS")


In [34]:
# Importing TransactionEncoder from mlxtend.preprocessing for binary encoding of the medications
from mlxtend.preprocessing import TransactionEncoder

# Converting 'GIVEN_MEDS' column values into a list of lists
aki_meds = meds_per_pt.GIVEN_MEDS.tolist()

# Initializing TransactionEncoder and applying fit and transform to the medication lists
te = TransactionEncoder()
te_ary = te.fit(aki_meds).transform(aki_meds)

# Creating a new DataFrame from the binary encoded array and naming the columns
df = pd.DataFrame(te_ary, columns=te.columns_)

In [35]:
# Adding 'PATIENT_DEID' and 'GIVEN_MEDS' columns back to the new DataFrame
df["PATIENT_DEID"] = meds_per_pt["PATIENT_DEID"]
df["GIVEN"] = meds_per_pt["GIVEN_MEDS"]

# Merging the binary encoded DataFrame with the selected_feature DataFrame on 'PATIENT_DEID'
vec_med_features = pd.merge(df, selected_feature, on="PATIENT_DEID")

In [37]:
vec_med_features.to_csv("vec_med_no_route_features_1029.csv", index = False)